# 퇴근후딴짓 빅데이터 분석기사 실기

In [1]:
# 라이브러리 불러오기
import pandas as pd
import numpy as np

### min-max 기준 상하위 5%값

* 주어진 데이터에서 'f5'컬럼을 min-max 스케일 변환한 후, 상위 5%와 하위 5% 값의 합을 구하시오

In [2]:
# 데이터 불러오기
df = pd.read_csv('C:/Users/chunc/Desktop/빅분기 연습/basic1.csv')
df.head()

,id,age,city,f1,f2,f3,f4,f5
0,id01,2.0,서울,NaN,0,NaN,ENFJ,91.297791
1,id02,9.0,서울,70.0,1,NaN,ENFJ,60.339826
2,id03,27.0,서울,61.0,1,NaN,ISTJ,17.252986
3,id04,75.0,서울,NaN,2,NaN,INFP,52.667078
4,id05,24.0,서울,85.0,2,NaN,ISFJ,29.269869


In [3]:
# EDA
print(df.isnull().sum())

id       0
age      0
city     0
f1      31
f2       0
f3      95
f4       0
f5       0
dtype: int64


In [4]:
# min-max 스케일링
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df['f5'] = scaler.fit_transform(df[['f5']])

# 방법 2
# df['f5'] = df['f5'].transform(lambda x: ((x - x.min()) / (x.max() - x.min())))

In [5]:
# 상위 5%와 하위 5% 값 구하기
lower = df['f5'].quantile(0.05)
print(lower)

upper = df['f5'].quantile(0.95)
print(upper)

print(lower + upper)

0.03670782406038746
0.9881662742993513
1.0248740983597389


### 상위 10개, 하위 10개 차이

* 주어진 데이터에서 상위 10개 국가의 접종률 평균과 하위 10개 국가의 접종률 평균을 구하고, 그 차이를 구해보세요
* (단, 100%가 넘는 접종률 제거, 소수 첫째자리까지 출력)

In [6]:
# 데이터 불러오기
df = pd.read_csv('C:/Users/chunc/Desktop/빅분기 연습/covid/covid-vaccination-vs-death_ratio.csv')
df.head()

,Unnamed: 0,country,iso_code,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,New_deaths,population,ratio
0,0,Afghanistan,AFG,2021-05-11,504502.0,448878.0,55624.0,12,40374668.0,1.111781
1,1,Afghanistan,AFG,2021-05-20,547901.0,470341.0,77560.0,10,40374668.0,1.164941
2,2,Afghanistan,AFG,2021-05-24,573277.0,476367.0,96910.0,10,40374668.0,1.179866
3,3,Afghanistan,AFG,2021-05-26,590454.0,479372.0,111082.0,19,40374668.0,1.187309
4,4,Afghanistan,AFG,2021-05-27,593313.0,479574.0,113739.0,14,40374668.0,1.187809


In [7]:
# 국가별 접종률 그룹화
#시간에 따라 접종률이 점점 올라감
df2 = df.groupby('country').max()
df2 = df2.sort_values(by = 'ratio', ascending = False)

In [8]:
# 100%가 넘는 접종률 제거
cond = df2['ratio'] <= 100
df2 = df2[cond]

top = df2['ratio'].head(10).mean()
bottom = df2['ratio'].tail(10).mean()

print(round(top - bottom, 1))

88.4


### 상관관계 구하기

* 주어진 데이터에서 상관관계를 구하고, quality와의 상관관계가 가장 큰 값과, 가장 작은 값을 구한 다음 더하시오!
* 단, quality와 quality 상관관계 제외, 소수점 둘째 자리까지 출력

In [9]:
df = pd.read_csv('C:/Users/chunc/Desktop/빅분기 연습/redwine/winequality-red.csv')
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [10]:
# 상관관계 구하기
df_corr = df.corr()
# quality-quality 제거
df_corr = df_corr[:-1]
print(df_corr['quality'])

fixed acidity           0.124052
volatile acidity       -0.390558
citric acid             0.226373
residual sugar          0.013732
chlorides              -0.128907
free sulfur dioxide    -0.050656
total sulfur dioxide   -0.185100
density                -0.174919
pH                     -0.057731
sulphates               0.251397
alcohol                 0.476166
Name: quality, dtype: float64


In [11]:
# 상관관계가 가장 큰 값과 가장 작은 값 (절대값으로 확인)
max_corr = abs(df.corr()['quality'][:-1]).max()
min_corr = abs(df.corr()['quality'][:-1]).min()

if max_corr not in df.corr()[['quality']][:-1].values:
    max_corr = -max_corr
if min_corr not in df.corr()[['quality']][:-1].values:
    min_corr = -min_corr

# 결과출력
ans = round(max_corr + min_corr, 2)
print(ans)


0.49


### 2개 조건에 따른 상위 값

* city와 f4를 기준으로 f5의 평균값을 구한 다음, f5를 기준으로 상위 7개 값을 모두 더해 출력하시오 (소수점 둘째자리까지 출력)

In [12]:
# 데이터 불러오기
df = pd.read_csv('C:/Users/chunc/Desktop/빅분기 연습/basic1.csv')
df.head()

,id,age,city,f1,f2,f3,f4,f5
0,id01,2.0,서울,NaN,0,NaN,ENFJ,91.297791
1,id02,9.0,서울,70.0,1,NaN,ENFJ,60.339826
2,id03,27.0,서울,61.0,1,NaN,ISTJ,17.252986
3,id04,75.0,서울,NaN,2,NaN,INFP,52.667078
4,id05,24.0,서울,85.0,2,NaN,ISFJ,29.269869


In [13]:
# city와 f4를 기준으로 f5의 평균값
new_df = df.groupby(['city', 'f4'])[['f5']].mean()

In [14]:
# f5를 기준으로 상위 7개 값을 모두 더해 출력
result = df.reset_index().sort_values('f5', ascending = False).head(7)
print(result)

    index    id   age city    f1  f2   f3    f4         f5
10     10  id11  40.0   서울  68.0   0  NaN  ENFP  98.429899
97     97  id98  39.0   경기  58.0   2  NaN  INFP  98.429899
9       9  id10  95.0   서울  74.0   1  NaN  ISFP  98.429899
76     76  id77  77.0   경기  31.0   0  NaN  INFP  98.429899
98     98  id99   1.0   경기  47.0   0  NaN  ESFJ  97.381034
91     91  id92  97.0   경기  78.0   1  NaN  INFP  97.381034
86     86  id87  19.0   경기   NaN   1  NaN  ISFP  97.381034


In [15]:
# 상위 7개 값을 모두 합하기
print(round(result['f5'].sum(), 2))

685.86


### 슬라이싱 & 조건

* 주어진 데이터 셋에서 age컬럼 상위 20개의 데이터를 구한 다음 f1의 결측치를 중앙값으로 채운다.
* 그리고 f4가 ISFJ와 f5가 20 이상인 f1의 평균값을 출력하시오

In [16]:
# 데이터 불러오기
df = pd.read_csv('C:/Users/chunc/Desktop/빅분기 연습/basic1.csv')
df.head()

,id,age,city,f1,f2,f3,f4,f5
0,id01,2.0,서울,NaN,0,NaN,ENFJ,91.297791
1,id02,9.0,서울,70.0,1,NaN,ENFJ,60.339826
2,id03,27.0,서울,61.0,1,NaN,ISTJ,17.252986
3,id04,75.0,서울,NaN,2,NaN,INFP,52.667078
4,id05,24.0,서울,85.0,2,NaN,ISFJ,29.269869


In [17]:
# age컬럼 상위 20개의 데이터를 구하기
df = df.sort_values('age', ascending = False).reset_index(drop = True)[:20]
print(df)

      id    age city    f1  f2   f3    f4         f5
0   id97  100.0   경기   NaN   0  NaN  INFP  67.886373
1   id37  100.0   부산   NaN   0  NaN  ESTP  33.308999
2   id45   97.0   대구  88.0   0  NaN  ENFJ  13.049921
3   id52   97.0   대구  82.0   1  NaN  ISFJ  90.496999
4   id92   97.0   경기  78.0   1  NaN  INFP  97.381034
5   id10   95.0   서울  74.0   1  NaN  ISFP  98.429899
6   id46   93.0   대구   NaN   0  NaN  ESTJ  67.886373
7   id78   92.0   경기  96.0   1  NaN  INTJ  69.730313
8   id96   92.0   경기  53.0   1  NaN  ENTJ  52.667078
9   id73   90.0   경기   NaN   1  NaN  ISFJ  73.586397
10  id21   90.0   부산   NaN   1  NaN  ISFP  29.269869
11  id88   89.0   경기  75.0   0  NaN  ESTJ  60.339826
12  id63   88.0   경기  86.0   1  NaN  ISFJ  73.586397
13  id66   87.0   경기   NaN   1  NaN  ISFP  17.252986
14  id38   87.0   부산   NaN   1  NaN  ESTP  83.685380
15  id61   87.0   경기  62.0   2  NaN  INTP  69.730313
16  id50   86.0   대구  78.0   1  NaN  ENFP  60.339826
17  id81   86.0   경기  50.0   1  NaN  ISFJ  37.

In [18]:
# f1결측치 중앙값으로 채우기
df['f1'] = df['f1'].fillna(df['f1'].median())

In [19]:
# f4가 ISFJ와 f5가 20 이상인 f1의 평균값 구하기
cond = (df['f4'] == 'ISFJ') & (df['f5'] >= 20)
print(df[cond]['f1'].mean())

73.875


### 분산

* 주어진 데이터 셋에서 f2가 0값인 데이터를 age를 기준으로 오름차순 정렬하고 앞에서 부터 20개의 데이터를 추출한 후 
* f1 결측치(최소값)를 채우기 전과 후의 분산 차이를 계산하시오 (소수점 둘째 자리까지)

In [20]:
# 데이터 불러오기
df = pd.read_csv('C:/Users/chunc/Desktop/빅분기 연습/basic1.csv')
df.head()

,id,age,city,f1,f2,f3,f4,f5
0,id01,2.0,서울,NaN,0,NaN,ENFJ,91.297791
1,id02,9.0,서울,70.0,1,NaN,ENFJ,60.339826
2,id03,27.0,서울,61.0,1,NaN,ISTJ,17.252986
3,id04,75.0,서울,NaN,2,NaN,INFP,52.667078
4,id05,24.0,서울,85.0,2,NaN,ISFJ,29.269869


In [21]:
# f2가 0값인 데이터를 age를 기준으로 오름차순 정렬
df = df[df['f2'] == 0].sort_values('age', ascending = True).reset_index(drop = True)

In [22]:
# 앞에서 부터 20개의 데이터를 추출
df = df[:20]

In [23]:
# f1 결측치(최소값)를 채우기 전과 후의 분산
df_var1 = df['f1'].var()
df['f1'] = df['f1'].fillna(df['f1'].min())
df_var2 = df['f1'].var()

In [24]:
# 소수점 둘째자리까지 출력
print(round(df_var1 - df_var2, 2))

38.44


### 시계열 데이터1

* 2022년 5월 sales의 중앙값을 구하시오

In [25]:
# 데이터 불러오기
df = pd.read_csv('C:/Users/chunc/Desktop/빅분기 연습/basic2.csv')
df.head()

,Date,Sales,PV,UV,Events
0,2022-01-01,22711525,397349,4421.0,1
1,2022-01-02,36779,3969,2289.0,0
2,2022-01-03,13943875,373890,NaN,1
3,2022-01-04,8088593,142054,2157.0,0
4,2022-01-05,2395374,190684,NaN,0


In [26]:
# datetime으로 type변경
df['Date'] = pd.to_datetime(df['Date'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 730 entries, 0 to 729
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    730 non-null    datetime64[ns]
 1   Sales   730 non-null    int64         
 2   PV      730 non-null    int64         
 3   UV      684 non-null    float64       
 4   Events  730 non-null    int64         
dtypes: datetime64[ns](1), float64(1), int64(3)
memory usage: 28.6 KB


In [27]:
# 새로운 컬럼 추가 (년, 월, 일)
df['year'] = df['Date'].dt.year
df['month'] = df['Date'].dt.month
df['day'] = df['Date'].dt.day

In [28]:
# 2022년 5월 sales의 중앙값
result = df[(df['year'] == 2022) & (df['month'] == 5)]
print(result['Sales'].median())

1477685.0


### 시계열 데이터2

* 주어진 데이터에서 2022년 5월 주말과 평일의 sales컬럼 평균값 차이를 구하시오 (소수점 둘째자리까지 출력, 반올림)

In [29]:
# 데이터 불러오기
df = pd.read_csv('C:/Users/chunc/Desktop/빅분기 연습/basic2.csv')
df.head()

,Date,Sales,PV,UV,Events
0,2022-01-01,22711525,397349,4421.0,1
1,2022-01-02,36779,3969,2289.0,0
2,2022-01-03,13943875,373890,NaN,1
3,2022-01-04,8088593,142054,2157.0,0
4,2022-01-05,2395374,190684,NaN,0


In [30]:
# datetime으로 type변경
df['Date'] = pd.to_datetime(df['Date'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 730 entries, 0 to 729
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    730 non-null    datetime64[ns]
 1   Sales   730 non-null    int64         
 2   PV      730 non-null    int64         
 3   UV      684 non-null    float64       
 4   Events  730 non-null    int64         
dtypes: datetime64[ns](1), float64(1), int64(3)
memory usage: 28.6 KB


In [31]:
# 새로운 컬럼 추가 (년, 월, 일)
df['year'] = df['Date'].dt.year
df['month'] = df['Date'].dt.month
df['day'] = df['Date'].dt.day
df['dayofweek'] = df['Date'].dt.dayofweek

In [32]:
# 주말 컬럼 생성
df['weekend'] = df['dayofweek'].apply(lambda x: x >= 5)

In [33]:
# 2022년 5월 주말과 평일 조건 생성
weekend_cond = (df['year'] == 2022) & (df['month'] == 5) & (df['weekend'])
weekday_cond = (df['year'] == 2022) & (df['month'] == 5) & (~df['weekend'])

In [34]:
# 2022년 5월 주말과 평일의 sales컬럼 평균값
weekend = df[weekend_cond]['Sales'].mean()
weekday = df[weekday_cond]['Sales'].mean()

In [35]:
# 평균값의 차이
print(round(weekend - weekday, 1))

3010339.1


### 시계열 데이터3

* 주어진 데이터에서 2022년 월별 Sales 합계 중 가장 큰 금액과 2023년 월별 Sales 합계 중 가장 큰 금액의 차이를 절대값으로 구하시오.
* 단 Events컬럼이 '1'인경우 80%의 Salse값만 반영함
* (최종값은 소수점 반올림 후 정수 출력)

In [36]:
# 데이터 불러오기
df = pd.read_csv('C:/Users/chunc/Desktop/빅분기 연습/basic2.csv')
df.head()

,Date,Sales,PV,UV,Events
0,2022-01-01,22711525,397349,4421.0,1
1,2022-01-02,36779,3969,2289.0,0
2,2022-01-03,13943875,373890,NaN,1
3,2022-01-04,8088593,142054,2157.0,0
4,2022-01-05,2395374,190684,NaN,0


In [37]:
# datetime으로 type변경
df['Date'] = pd.to_datetime(df['Date'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 730 entries, 0 to 729
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    730 non-null    datetime64[ns]
 1   Sales   730 non-null    int64         
 2   PV      730 non-null    int64         
 3   UV      684 non-null    float64       
 4   Events  730 non-null    int64         
dtypes: datetime64[ns](1), float64(1), int64(3)
memory usage: 28.6 KB


In [38]:
# 새로운 컬럼 추가 (년, 월, 일)
df['year'] = df['Date'].dt.year
df['month'] = df['Date'].dt.month
df['day'] = df['Date'].dt.day
df['dayofweek'] = df['Date'].dt.dayofweek

In [39]:
# Events컬럼이 '1'인경우 80%의 Salse값만 반영
def event_sales(x):
    if x['Events'] == 1:
        x['Sales'] = x['Sales'] * 0.8
    else:
        x['Sales'] = x['Sales']
    return x

df = df.apply(lambda x: event_sales(x), axis = 1)
df.head()

,Date,Sales,PV,UV,Events,year,month,day,dayofweek
0,2022-01-01,18169220.0,397349,4421.0,1,2022,1,1,5
1,2022-01-02,36779.0,3969,2289.0,0,2022,1,2,6
2,2022-01-03,11155100.0,373890,NaN,1,2022,1,3,0
3,2022-01-04,8088593.0,142054,2157.0,0,2022,1,4,1
4,2022-01-05,2395374.0,190684,NaN,0,2022,1,5,2


In [40]:
# 2022년 월별 Sales 합계 중 가장 큰 금액
df1 = df[df['year'] == 2022]
sale1 = df1.groupby('month')['Sales'].sum().max()
sale1

197145878.0

In [41]:
# 2023년 월별 Sales 합계 중 가장 큰 금액
df2 = df[df['year'] == 2023]
sale2 = df2.groupby('month')['Sales'].sum().max()
sale2

239619313.8

In [42]:
# 금액의 차이를 절대값 정수로 출력
print(int(round(abs(sale1 - sale2), 0)))

42473436


### 데이터 병합

* basic1 데이터와 basic3 데이터를 'f4'값을 기준으로 병합하고, 병합한 데이터에서 r2결측치를 제거한 다음, 앞에서 부터 20개 데이터를 선택하고 'f2'컬럼 합을 구하시오

In [43]:
# 데이터 불러오기
b1 = pd.read_csv('C:/Users/chunc/Desktop/빅분기 연습/basic1.csv')
b3 = pd.read_csv('C:/Users/chunc/Desktop/빅분기 연습/basic3.csv')

In [44]:
# 두 데이터를 'f4'값 기준으로 병합
df = pd.merge(left = b1, right = b3, how = 'left', on = 'f4')
df.head()

,id,age,city,f1,f2,f3,f4,f5,r1,r2
0,id01,2.0,서울,NaN,0,NaN,ENFJ,91.297791,INFP,ISFP
1,id02,9.0,서울,70.0,1,NaN,ENFJ,60.339826,INFP,ISFP
2,id03,27.0,서울,61.0,1,NaN,ISTJ,17.252986,ESFP,NaN
3,id04,75.0,서울,NaN,2,NaN,INFP,52.667078,ENFJ,ENTJ
4,id05,24.0,서울,85.0,2,NaN,ISFJ,29.269869,ESFP,ESTP


In [45]:
# 결측치 확인
df.isnull().sum()

id       0
age      0
city     0
f1      31
f2       0
f3      95
f4       0
f5       0
r1       0
r2      17
dtype: int64

In [46]:
# r2 결측치 제거
print(df.shape)
df = df.dropna(subset = ['r2'])
print(df.shape)

(100, 10)
(83, 10)


In [47]:
# 인덱스 리셋
df = df.reset_index()

In [48]:
# 앞에서 부터 20개 데이터를 선택하고 'f2'컬럼 합 구하기
print(df.iloc[:20]['f2'].sum())

15


### 구간 분할

* basic1 데이터 중 'age'컬럼 이상치를 제거하고, 동일한 개수로 나이 순으로 3그룹으로 나눈 뒤 각 그룹의 중앙값을 더하시오
* (이상치는 음수(0포함), 소수점 값)

In [49]:
# 데이터 불러오기
df = pd.read_csv('C:/Users/chunc/Desktop/빅분기 연습/basic1.csv')
df.head()

,id,age,city,f1,f2,f3,f4,f5
0,id01,2.0,서울,NaN,0,NaN,ENFJ,91.297791
1,id02,9.0,서울,70.0,1,NaN,ENFJ,60.339826
2,id03,27.0,서울,61.0,1,NaN,ISTJ,17.252986
3,id04,75.0,서울,NaN,2,NaN,INFP,52.667078
4,id05,24.0,서울,85.0,2,NaN,ISFJ,29.269869


In [50]:
# 'age'컬럼 이상치를 제거(이상치는 음수(0포함))
df = df[~(df['age'] <= 0)]

# 소수점 제거
df = df[(df['age'] == round(df['age'], 0))]

In [51]:
# 구간분할 및 수량 비교
df['range'] = pd.qcut(df['age'], q = 3, labels = ['group1', 'group2', 'group3'])
df['range'].value_counts()

range
group1    30
group2    30
group3    30
Name: count, dtype: int64

In [52]:
# 각 그룹의 중앙값 더하기
g1_med = df[df['range'] == 'group1']['age'].median()
g2_med = df[df['range'] == 'group2']['age'].median()
g3_med = df[df['range'] == 'group3']['age'].median()
print(g1_med + g2_med + g3_med)

165.0


### Time-Series4

* 주어진 데이터(basic2.csv)에서 주 단위 Sales의 합계를 구하고, 가장 큰 값을 가진 주와 작은 값을 가진 주의 차이를 구하시오(절대값)

In [53]:
# 데이터 불러오기
df = pd.read_csv('C:/Users/chunc/Desktop/빅분기 연습/basic2.csv', parse_dates=['Date'], index_col = 0)
df.head()

,Sales,PV,UV,Events
Date,,,,
2022-01-01,22711525,397349,4421.0,1
2022-01-02,36779,3969,2289.0,0
2022-01-03,13943875,373890,NaN,1
2022-01-04,8088593,142054,2157.0,0
2022-01-05,2395374,190684,NaN,0


단위
* 주 단위 W
* 2주 단위 2W
* 월 단위 M

In [54]:
# 주 단위 Sales의 합계
df_w = df.resample('W').sum()
df_w.head()

,Sales,PV,UV,Events
Date,,,,
2022-01-02,22748304,401318,6710.0,1
2022-01-09,33373822,1148464,15189.0,2
2022-01-16,20754519,283773,9344.0,0
2022-01-23,21880751,538633,9841.0,1
2022-01-30,28878452,560814,7808.0,1


In [55]:
# 가장 큰 값을 가진 주와 작은 값을 가진 주 차이값
ma = df_w['Sales'].max()
mi = df_w['Sales'].min()
print(ma - mi)

91639050


### 중복 데이터 제거

f1의 결측치를 채운 후 age 컬럼의 중복 제거 전과 후의 'f1' 중앙값 차이를 구하시오
- 결측치는 f1의 데이터 중 10번째로 큰 값으로 채움
- 중복 데이터 발생시 뒤에 나오는 데이터를 삭제함
- 최종 결과값은 절대값으로 출력

In [56]:
# 데이터 불러오기
df = pd.read_csv('C:/Users/chunc/Desktop/빅분기 연습/basic1.csv')
df.tail()

,id,age,city,f1,f2,f3,f4,f5
95,id96,92.0,경기,53.0,1,NaN,ENTJ,52.667078
96,id97,100.0,경기,NaN,0,NaN,INFP,67.886373
97,id98,39.0,경기,58.0,2,NaN,INFP,98.429899
98,id99,1.0,경기,47.0,0,NaN,ESFJ,97.381034
99,id100,47.0,경기,53.0,0,vip,ESFP,33.308999


In [57]:
# f1의 데이터 중 10번째로 큰 값으로 채움
top10 = df['f1'].sort_values(ascending = False).iloc[9]
df['f1'] = df['f1'].fillna(top10)

In [58]:
# 중복 제거전 중앙값
result1 = df['f1'].median()

In [59]:
# 중복제거
df = df.drop_duplicates(subset = ['age'])
print(df.shape)

(71, 8)


In [60]:
# 중복 제거후 중앙값
result2 = df['f1'].median()

In [61]:
# 차이값(절대값)
print(abs(result1 - result2))

0.5


### Time-Series5 (Lagged Feature) 시차 데이터 생성

* 주어진 데이터(basic2.csv)에서 "pv"컬럼으로 1일 시차(lag)가 있는 새로운 컬럼을 만들고(예: 1월 2일에는 1월 1일 pv데이터를 넣고, 1월 3일에는 1월 2일 pv데이터를 넣음),새로운 컬럼의 1월 1일은 다음날(1월 2일)데이터로 결측치를 채운 다음, Events가 1이면서 Sales가 1000000이하인 조건에 맞는 새로운 컬럼 합을 구하시오

In [62]:
# 데이터 불러오기
df = pd.read_csv('C:/Users/chunc/Desktop/빅분기 연습/basic2.csv')
df.tail()

,Date,Sales,PV,UV,Events
725,2023-12-27,33846609,415067,7360.0,1
726,2023-12-28,3659566,208601,2262.0,1
727,2023-12-29,1164008,14309,193.0,0
728,2023-12-30,3977696,60935,NaN,0
729,2023-12-31,3719764,60963,1507.0,0


In [63]:
# 결측치 확인
df.isnull().sum()

Date       0
Sales      0
PV         0
UV        46
Events     0
dtype: int64

In [64]:
# 1차이가 나는 시차 특성 만들기
df['previous_PV'] = df['PV'].shift(1)
df.head()

,Date,Sales,PV,UV,Events,previous_PV
0,2022-01-01,22711525,397349,4421.0,1,NaN
1,2022-01-02,36779,3969,2289.0,0,397349.0
2,2022-01-03,13943875,373890,NaN,1,3969.0
3,2022-01-04,8088593,142054,2157.0,0,373890.0
4,2022-01-05,2395374,190684,NaN,0,142054.0


In [65]:
# 새로운 컬럼의 1월 1일은 다음날(1월 2일)데이터로 결측치를 채우기
df['previous_PV'] = df['previous_PV'].fillna(method = 'bfill')
df.head()

,Date,Sales,PV,UV,Events,previous_PV
0,2022-01-01,22711525,397349,4421.0,1,397349.0
1,2022-01-02,36779,3969,2289.0,0,397349.0
2,2022-01-03,13943875,373890,NaN,1,3969.0
3,2022-01-04,8088593,142054,2157.0,0,373890.0
4,2022-01-05,2395374,190684,NaN,0,142054.0


In [66]:
# Events가 1이면서 Sales가 1000000이하인 조건에 맞는 새로운 컬럼 합
result = (df['Events'] == 1) & (df['Sales'] <= 1000000)
print(df[result]['previous_PV'].sum())

1894876.0
